In [ ]:
import python
import pandas as pd 
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "YourKey")

## Creating The Bryan Aguirre Twitter Parody 


For this project I thought it would be fun to create a parody account mimicking and predicting funny and sarcastic tweets from myself. 


In this notebook we'll look at the prompting technique I used. 


and in the full code you'll see the other helper code used to deploy this app.

# Here's the Gist of how it will work:

A user @mentions your bot, for me it will be @b_ryanaguirre_ or Bryan Aguirre (Parody)

The script starts by finding a new @mention and then reads the parent tweet it is being mentioned on

The script will then take that parent tweet and generates a witty response using a language model

Respond is posted and tweet is logged

# Creating our LLM

How we train our model, 

we start with creating a function that will take in/feed in my tweet, and give an output response.

we are making sure it is under 200 characters and creating around a 5-10 min response window given the norm behind twitter limits.

In [ ]:
llm = ChatOpenAI(temperature=0.3,
                 openai_api_key=OPENAI_API_KEY,
#                  model_name='gpt-3.5-turbo',
                 model_name='gpt-4',
                )
                

# Prompt Editing Process


Prompt 

You are an incredibly wise, and smart, sarcastic, comedian. Your goal is to give a concise prediction in response to a piece of text from Bryan Aguirre.


% RESPONSE TONE:

Your prediction should be given in an active voice and be opinionated
Your tone should be serious w/ a hint of wit and sarcasm

% RESPONSE FORMAT:

Respond in under 200 characters
Respond in one short sentence
Do not respond with emojis

% RESPONSE CONTENT:

Include specific examples if they are relevant
If you don't have an answer, say, "Sorry, nevermind "


In [ ]:
def generate_response(llm, mentioned_parent_tweet_text):
    # It would be nice to bring in information about the links, pictures, etc.
    # But out of scope for now
    
    system_template = """
        You are an incredibly wise and smart tech mad scientist from silicon valley.
        Your goal is to give a concise prediction in response to a piece of text from the user.
        
        % RESPONSE TONE:

        - Your prediction should be given in an active voice and be opinionated
        - Your tone should be serious w/ a hint of wit and sarcasm
        
        % RESPONSE FORMAT:

        - Respond in under 200 characters
        - Respond in two or less short sentences
        - Do not respond with emojis
        
        % RESPONSE CONTENT:

        - Include specific examples of old tech if they are relevant
        - If you don't have an answer, say, "Sorry, my magic 8 ball isn't working right now 🔮"
    """
    system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

    human_template="{text}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    # get a chat completion from the formatted messages
    final_prompt = chat_prompt.format_prompt(text=mentioned_parent_tweet_text).to_messages()
    response = llm(final_prompt).content

    return response

In [ ]:
tweet = """
I wanted to build a sassy Twitter Bot Bryan Aguirre (Parody) that responds, parodies, and predicts tweets by Bryan Aguirre 

@b_ryanaguirre_ was built using @LangChainAI and hosted on @railway 

Condensed Prompt:
You are an incredibly wise, and smart, sarcastic, comedian. Your goal is to give a concise prediction in response to a piece of text from Bryan.
"""

response = generate_response(llm, tweet)
print (response)